# A Patchy work on Individual Stock Data

In [1]:
import os
import pandas as pd
import pickle
import numpy as np
import math

from collections import Counter
from nsetools import Nse
from nsepy.history import get_history

from sklearn import svm, neighbors 
from sklearn import preprocessing

from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [2]:
def save_nse_200_tickers():
    list_of_stock_codes = list(pd.read_csv('nifty_list/nifty200list.csv')['Symbol'])
    return list_of_stock_codes

## Let's Look on IndusInd bank data 

In [3]:
df = pd.read_csv('nse_data_new_500/{}.csv'.format('INDUSINDBK'))

In [4]:
df_new = df.loc[:,[ 'Close', 'Volume']]
df_new['HL_PCT'] = (df['High'] - df['Low']) / df['Close'] * 100.0
df_new['PCT_change'] = (df['Close'] - df['Open']) / df['Open'] * 100.0

In [7]:
# df_new

In [6]:
# Drop missing value
df_new.fillna(value=-99999, inplace=True)

# We want to separate 1 percent of the data to forecast
forecast_out = int(math.ceil(0.01 * len(df_new)))

# Separating the label here, we want to predict the AdjClose
forecast_col = 'Close'

df_new['label'] = df_new[forecast_col].shift(-forecast_out)

X = np.array(df_new.drop(['label'], 1))

# Scale the X so that everyone can have the same distribution for linear regression
X = preprocessing.scale(X)

# Finally We want to find Data Series of late X and early X (train) for model generation and evaluation
X_lately = X[-forecast_out:]
X = X[:-forecast_out]

# Separate label and identify it as y
y = np.array(df_new['label'])
y = y[:-forecast_out]


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Linear regression
clfreg = LinearRegression(n_jobs=-1)
clfreg.fit(X_train, y_train)

# Quadratic Regression 2
clfpoly2 = make_pipeline(PolynomialFeatures(2), Ridge())
clfpoly2.fit(X_train, y_train)

# Quadratic Regression 3
clfpoly3 = make_pipeline(PolynomialFeatures(3), Ridge())
clfpoly3.fit(X_train, y_train)

# KNN Regression
clfknn = KNeighborsRegressor(n_neighbors=2)
clfknn.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                    weights='uniform')

## Results

In [9]:
confidencereg = clfreg.score(X_test, y_test)
confidencepoly2 = clfpoly2.score(X_test,y_test)
confidencepoly3 = clfpoly3.score(X_test,y_test)
confidenceknn = clfknn.score(X_test, y_test)

In [10]:
clfreg.score(X_test, y_test)

0.9379173694695087

In [11]:
clfpoly2.score(X_test,y_test)

0.9112385111576801

In [12]:
clfpoly3.score(X_test,y_test)

0.9313549377709206

In [13]:
clfknn.score(X_test, y_test)

0.877559190504006